In [1]:
# Importar las librerías necesarias
import os
import numpy as np
import requests
import csv

In [48]:
# Definir la función para leer y procesar el archivo .txt
def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
    
    # Encontrar el índice donde termina el encabezado
    end_of_header = 0
    for i, line in enumerate(lines):
        if line.strip() == '# EndOfHeader':
            end_of_header = i
            break
    
    # Leer los datos desde el final del encabezado
    data_lines = lines[end_of_header + 1:]
    data = [list(map(int, line.strip().split('\t'))) for line in data_lines]
    return np.array(data)

# Ruta del archivo .txt
txt_file_path = 'respondiendo_preguntas_12seg.txt'

# Leer los datos del archivo
data = read_txt_file(txt_file_path)

In [49]:
# Identificar la señal que más varía (columna con la mayor desviación estándar)
def find_most_varying_signal(data):
    variances = np.var(data, axis=0)
    most_varying_signal_index = np.argmax(variances)
    return most_varying_signal_index

most_varying_signal_index = find_most_varying_signal(data)
print(f'La señal que más varía es la columna {most_varying_signal_index}')

La señal que más varía es la columna 8


In [50]:
# Ventaneo de los datos y creación de archivos CSV
def create_csv_windows(data, signal_index, window_size, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    num_windows = len(data) // window_size
    for i in range(num_windows):
        window_data = data[i * window_size:(i + 1) * window_size, signal_index]
        output_file = os.path.join(output_folder, f'EMG.sig_{i}.csv')
        with open(output_file, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow(['timestamp', 'signal'])
            for j, value in enumerate(window_data):
                writer.writerow([j, value])

# Definir el tamaño de la ventana y la carpeta de salida
Fs = 1000
window_size = 2*Fs  # Como hay mucha duración, tomaré como ventaneo 2 segundos
output_folder = 'output_windows_EEG_PREGUNTAS'

# Crear los archivos CSV por ventana
create_csv_windows(data, most_varying_signal_index, window_size, output_folder)
print(f'Se han creado los archivos CSV en la carpeta {output_folder}')

Se han creado los archivos CSV en la carpeta output_windows_EEG_PREGUNTAS


In [51]:
# Subir los archivos CSV generados a Edge Impulse
def upload_files_to_edge_impulse(api_key, label, folder_path):
    url = 'https://ingestion.edgeimpulse.com/api/training/files'
    headers = {
        'x-api-key': api_key,
        'x-label': label,
    }
    files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.csv')]
    for file_path in files:
        with open(file_path, 'rb') as file:
            files_payload = {'data': (os.path.basename(file_path), file, 'application/csv')}
            res = requests.post(url, headers=headers, files=files_payload)
            if res.status_code == 200:
                print(f'Uploaded {file_path} to Edge Impulse successfully.')
            else:
                print(f'Failed to upload {file_path} to Edge Impulse. Status code: {res.status_code}')

# Definir la clave de API y la etiqueta
api_key = 'ei_51b8790275170673d369726381038ceadcc54546810ff47d8eb0e78978783422'
label = 'eeg_preguntas'

# Subir los archivos a Edge Impulse
upload_files_to_edge_impulse(api_key, label, output_folder)

Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_0.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_1.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_10.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_11.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_12.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_13.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_14.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_15.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_16.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_17.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_18.csv to Edge Impulse successfully.
Uploaded output_windows_EEG_PREGUNTAS\EMG.sig_19.csv to Edge Impulse successfully.
Upload